# Portfolio Performance vs. Benchmark

### Step 1: Import Libraries

In [1]:
# Data manipulation libraries
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay

# Visualization libraries
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import Image, display

# System libraries
import os
import sys
import glob
import logging
import warnings

warnings.filterwarnings("ignore", category=UserWarning)  # Font warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=".*inplace.*")  # Pandas warnings
logging.getLogger('matplotlib.font_manager').disabled = True

In [2]:
from py.quantstats_fix import *
from py.utils import load_and_filter_data, export_to_excel
qs.extend_pandas()

               QuantStats Compatibility Tool                

Part 1: Directly patching QuantStats package files
------------------------------------------------------------
Found QuantStats utils file at: /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/__init__.py
Successfully fixed indentation in QuantStats __init__.py file
✓ QuantStats utils file patched successfully

Part 2: Fixing resampling issues
------------------------------------------------------------
Found 1 potential QuantStats installation(s)
Checking /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py
✓ Found 'plot_timeseries' function in /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py
✓ No 'sum(axis=0)' calls found - may already be fixed
Examining /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py...
✓ Found 'plot_timeserie

### Step 2: Define Parameters 

#### Dates

In [3]:
# Define the date range
end_date = (datetime.today() - BDay(1)).to_pydatetime()  # Subtract 1 business day
# end_date = pd.to_datetime('2025-04-26')  # Report date
start_date = end_date - timedelta(days=10*365)

# Convert datetime objects to Unix timestamps (seconds since Jan 1, 1970)
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

# Print the date range
days_difference = (end_date - start_date).days
print(f"Date Range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"Time span: {days_difference} days ({days_difference/365:.2f} years)")

Date Range: 2015-06-09 to 2025-06-06
Time span: 3650 days (10.00 years)


#### Report File

In [4]:
# Get the most recent portfolio file
portfolio_files = glob.glob('portfolios/portfolio-*.xlsx')
if portfolio_files:
    # Sort files by modification time (most recent first)
    output_file = max(portfolio_files, key=os.path.getmtime)
    print(f"Using most recent portfolio file: {output_file}")
else:
    # Fallback to current date if no files found
    output_file = f'portfolios/portfolio-{datetime.date(end_date)}.xlsx'
    print(f"No portfolio files found. Using: {output_file}")

Using most recent portfolio file: portfolios/portfolio-2025-06-06.xlsx


#### Benchmark

In [5]:
# benchmark = 'ADME'
benchmark = pd.read_excel(output_file, sheet_name="benchmark_short_term")['Benchmark'].values[0]
print(benchmark)

DSI


#### Risk-free rate (T-bill, %)

In [6]:
# Load and process data
risk_free_df = pd.read_excel(output_file, sheet_name="daily_quotes", index_col=0)['^IRX']
# risk_free_rate  = 0.0433                              # 3‑month T‑Bill
risk_free_rate = risk_free_df.iloc[-1] / 100 

# Display result
print("Risk-Free Rate:", risk_free_rate, "-- 13 WEEK TREASURY BILL (^IRX)")

Risk-Free Rate: 0.0424 -- 13 WEEK TREASURY BILL (^IRX)


### Step 3: Read Portfolio Data (Excel)

In [7]:
portfolio_df = pd.read_excel(output_file, sheet_name="short_term")

# Convert percentage strings to float values
portfolio_df['Weight'] = portfolio_df['Weight'].replace('%', '', regex=True).astype(float)
portfolio_tickers = portfolio_df["Ticker"].tolist()

print(portfolio_tickers)
display(portfolio_df)

['GRMN', 'MSFT']


,Ticker,Date,Name,Sector,Industry,Country,Website,Market Cap,Enterprise Value,Float Shares,...,Short Period,Long Period,Articles In Last Week,Company News Score,Bearish Percent,Bullish Percent,Average Sentiment Score,Sector Average Bullish Percent,Sector Average News Score,Weight
0,GRMN,2025-06-02,Garmin Ltd.,Technology,Scientific & Technical Instruments,Switzerland,https://www.garmin.com,39080046592,36584062976,174794910,...,10,190,6,0.6083,0.0000,0.3333,0.2167,0.4831,0.566,0.5
1,MSFT,2025-06-02,Microsoft Corporation,Technology,Software - Infrastructure,United States,https://www.microsoft.com,3421643997184,3447046799360,7422063978,...,30,190,41,0.5961,0.0488,0.6098,0.1922,0.4831,0.566,0.5


### Step 4: Download Returns

In [8]:
# stock_returns = qs.utils.download_returns(ticker=portfolio_tickers, period="5y").dropna()
stock_quotes = load_and_filter_data('data/daily_stock_quotes.csv', portfolio_tickers, start_date, end_date)
stock_returns = np.log(stock_quotes / stock_quotes.shift(1)).dropna()

benchmark_quotes = load_and_filter_data('data/daily_benchmark_quotes.csv', benchmark, start_date, end_date)
benchmark_returns = np.log(benchmark_quotes / benchmark_quotes.shift(1)).dropna()

# Display summary statistics for all assets
print("\nSharpe Ratios for individual assets:")
sharpes = {col: qs.stats.sharpe(stock_returns[col]) for col in stock_returns.columns}
for ticker, sharpe in sharpes.items():
    print(f"{ticker}: {sharpe:.4f}")

display(stock_returns.head())

Found 2 of 2 tickers in data/daily_stock_quotes.csv
Missing tickers: []
Found 1 of 1 tickers in data/daily_benchmark_quotes.csv
Missing tickers: []

Sharpe Ratios for individual assets:
GRMN: 0.6344
MSFT: 0.9002


,GRMN,MSFT
Date,,
2015-06-11,-0.008790,-0.003718
2015-06-12,-0.016019,-0.010233
2015-06-15,-0.013245,-0.010846
2015-06-16,0.023657,0.007831
2015-06-17,-0.000888,0.003015


### Step 6: Split Data into Training and Testing Sets

In [9]:
# Split data into 80% training and 20% testing based on date
total_rows = len(stock_quotes)
train_size = int(0.8 * total_rows)
training_set = stock_quotes.iloc[:train_size]
test_set = stock_quotes.iloc[train_size:]

print(f"Training: {len(training_set)} rows ({len(training_set)/total_rows:.1%}) | Testing: {len(test_set)} rows ({len(test_set)/total_rows:.1%})")
print(f"Training period: {training_set.index[0]} to {training_set.index[-1]}")
print(f"Testing period: {test_set.index[0]} to {test_set.index[-1]}")

Training: 2009 rows (80.0%) | Testing: 503 rows (20.0%)
Training period: 2015-06-10 00:00:00 to 2023-06-01 00:00:00
Testing period: 2023-06-02 00:00:00 to 2025-06-04 00:00:00


### Step 5: Plot Return Comparisons

In [10]:
# Calculate portfolio returns using weights from Excel file
portfolio_weights = portfolio_df.set_index('Ticker')['Weight'].to_dict()
weighted_returns = pd.DataFrame()

print("\nPortfolio Weights:")
for ticker in portfolio_tickers:
    weight = portfolio_weights.get(ticker, 0)
    print(f"{ticker}: {weight:.2%}")
    if ticker in stock_returns.columns:
        weighted_returns[ticker] = stock_returns[ticker] * weight

# Sum across all weighted returns to get the portfolio return
portfolio_return = weighted_returns.sum(axis=1)

# Create equal-weight portfolio for comparison
equal_weight = 1/len([t for t in portfolio_tickers if t in stock_returns.columns])
equal_weighted_returns = pd.DataFrame()

for ticker in portfolio_tickers:
    if ticker in stock_returns.columns:
        equal_weighted_returns[ticker] = stock_returns[ticker] * equal_weight
        
equal_weight_return = equal_weighted_returns.sum(axis=1)

print("\nPortfolio Performance Summary:")
print(f"Sharpe Ratio (Weighted Portfolio): {qs.stats.sharpe(portfolio_return):.4f}")
print(f"Sharpe Ratio (Equal-Weight): {qs.stats.sharpe(equal_weight_return):.4f}")

plt.figure(figsize=(12, 6))
(1 + portfolio_return).cumprod().plot(label='Weighted Portfolio')
(1 + equal_weight_return).cumprod().plot(label='Equal Weight')
(1 + benchmark_returns).cumprod().plot(label=benchmark)
plt.legend()
plt.title('Performance Comparison')
plt.ylabel('Cumulative Return')
plt.grid(True)
plt.show()


Portfolio Weights:
GRMN: 50.00%
MSFT: 50.00%

Portfolio Performance Summary:
Sharpe Ratio (Weighted Portfolio): 0.8982
Sharpe Ratio (Equal-Weight): 0.8982


### Step 6: Generate Reports

In [11]:
import bt
import matplotlib.pyplot as plt

# Create weights dictionary using portfolio_df instead of optimal_portfolio_df
weights = {}
for _, row in portfolio_df.iterrows():
    ticker = row['Ticker']
    weight = row['Weight'] 
    weights[ticker] = weight

print("Portfolio weights:")
for ticker, weight in weights.items():
    print(f"{ticker}: {weight:.4f}")

# Pre-assign colors to tickers for consistency across all plots
ticker_color_map = {}
base_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', 
               '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#aec7e8', '#ffbb78']

for i, ticker in enumerate(portfolio_tickers):
    ticker_color_map[ticker] = base_colors[i % len(base_colors)]

print("\nTicker color assignments:")
for ticker, color in ticker_color_map.items():
    print(f"{ticker}: {color}")

# Create price data for bt (convert from quotes to prices)
# Combine stock quotes and benchmark quotes
all_quotes = test_set.copy()
all_quotes[benchmark] = benchmark_quotes[benchmark]

# Create bt strategies
# Strategy 1 - Portfolio with quarterly rebalancing
portfolio_strategy = bt.Strategy('Portfolio', 
                                [
                                    bt.algos.RunWeekly(),  # Trading every 66 days (approx. quarterly)                 
                                    bt.algos.SelectAll(),
                                    bt.algos.WeighSpecified(**weights),       
                                    bt.algos.Rebalance()
                                ])

# Strategy 2 - Benchmark (buy and hold)
benchmark_strategy = bt.Strategy(f'{benchmark}',
                                [
                                    bt.algos.RunWeekly(),
                                    bt.algos.SelectThese([benchmark]),
                                    bt.algos.WeighEqually(),
                                    bt.algos.Rebalance()
                                ])

# Create backtests
portfolio_test = bt.Backtest(portfolio_strategy, all_quotes[portfolio_tickers])
benchmark_test = bt.Backtest(benchmark_strategy, all_quotes[[benchmark]])

# Run backtests
result = bt.run(portfolio_test, benchmark_test)
result.set_riskfree_rate(risk_free_rate)

# Show performance metrics
result.display()
display(result.get_transactions())

# Plot weights over time using pre-assigned colors
fig, ax = plt.subplots(figsize=(10, 6))

# Get the weight data from the result - access it through the strategy result
portfolio_result = result['Portfolio Strategy']

ax.set_title('Portfolio Weights Over Time', fontsize=14, fontweight='bold')
ax.set_ylabel('Weight', fontsize=12)
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Plot cumulative returns comparison
result.plot(figsize=(12, 8),
           ylabel='Cumulative Returns',
           title='Portfolio vs Benchmark Performance',
           legend=True,
           grid=True)

Portfolio weights:
GRMN: 0.5000
MSFT: 0.5000

Ticker color assignments:
GRMN: #1f77b4
MSFT: #ff7f0e


100%|██████████| 2/2 [00:00<00:00,  7.73it/s]


Stat                 Portfolio    DSI
-------------------  -----------  ----------
Start                2023-06-01   2023-06-01
End                  2025-06-04   2025-06-04
Risk-free rate       4.24%        4.24%

Total Return         72.47%       40.70%
Daily Sharpe         1.11         0.84
Daily Sortino        2.06         1.40
CAGR                 31.16%       18.52%
Max Drawdown         -21.27%      -20.58%
Calmar Ratio         1.46         0.90

MTD                  0.87%        0.92%
3m                   6.40%        5.84%
6m                   1.74%        -2.65%
YTD                  5.80%        1.70%
1Y                   22.13%       11.89%
3Y (ann.)            31.16%       18.52%
5Y (ann.)            -            -
10Y (ann.)           -            -
Since Incep. (ann.)  31.16%       18.52%

Daily Sharpe         1.11         0.84
Daily Sortino        2.06         1.40
Daily Mean (ann.)    29.97%       18.56%
Daily Vol (ann.)     23.22%       17.07%
Daily Skew           1.64  

price  quantity
Date       Security                  
2023-06-02 GRMN      102.26    4889.0
           MSFT      330.26    1513.0
2023-06-05 GRMN      101.35      26.0
           MSFT      330.80      -8.0
2023-06-12 GRMN      102.15     -49.0
...                     ...       ...
2025-05-19 MSFT      458.87      10.0
2025-05-27 GRMN      205.51       2.0
           MSFT      460.69      -1.0
2025-06-02 GRMN      202.95      32.0
           MSFT      461.97     -14.0

[211 rows x 2 columns]

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


<Axes: title={'center': 'Portfolio vs Benchmark Performance'}, ylabel='Cumulative Returns'>

#### Portfolio (Weighted) vs Benchmark

In [12]:
# Filter portfolio_return and benchmark_returns to match test_set dates
test_start_date = test_set.index[0]
test_end_date = test_set.index[-1]

# Filter portfolio_return to test period
portfolio_return_test = portfolio_return.loc[test_start_date:test_end_date]
portfolio_return_test.name = "Weighted Portfolio"

# Filter benchmark_returns to test period
benchmark_returns_test = benchmark_returns.loc[test_start_date:test_end_date]

print(f"Original portfolio_return length: {len(portfolio_return)}")
print(f"Filtered portfolio_return length: {len(portfolio_return_test)}")
print(f"Test period: {test_start_date} to {test_end_date}")

# Generate Report using filtered data
qs.reports.html(
    portfolio_return_test,
    benchmark_returns_test,
    rf=risk_free_rate,
    figsize=(8, 5),
    output=f'portfolios/portfolio_vs_benchmark_short_term-{datetime.date(end_date)}.html',
    title=f'Portfolio vs {benchmark} (Benchmark) - Test Period',
    benchmark_title=f'{benchmark}',
    download_filename=f'portfolio_vs_{benchmark}.html'
)

qs.reports.full(
    portfolio_return_test, 
    benchmark_returns_test,
    rf=risk_free_rate, 
    figsize=(8, 5), 
    title=f'Portfolio vs {benchmark} - Test Period',
    benchmark_title=f'{benchmark}')

Original portfolio_return length: 2511
Filtered portfolio_return length: 503
Test period: 2023-06-02 00:00:00 to 2025-06-04 00:00:00


/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retai

Added download button and removed QuantStats attribution from portfolios/portfolio_vs_benchmark_short_term-2025-06-06.html


/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retai

                           DSI         Strategy
-------------------------  ----------  ----------
Start Period               2023-06-02  2023-06-02
End Period                 2025-06-04  2025-06-04
Risk-Free Rate             4.24%       4.24%
Time in Market             100.0%      100.0%

Cumulative Return          38.84%      61.97%
CAGR﹪                     0.04%       0.07%

Sharpe                     0.81        0.99
Prob. Sharpe Ratio         70.44%      76.55%
Smart Sharpe               0.73        0.9
Sortino                    1.17        1.55
Smart Sortino              1.06        1.41
Sortino/√2                 0.82        1.1
Smart Sortino/√2           0.75        1.0
Omega                      1.21        1.21

Max Drawdown               -21.06%     -22.15%
Longest DD Days            132         105
Volatility (ann.)          17.04%      22.87%
R^2                        0.51        0.51
Information Ratio          0.03        0.03
Calmar                     0.57        0.81

None

,Start,Valley,End,Days,Max Drawdown,99% Max Drawdown
1,2025-02-20,2025-04-08,2025-06-04,105,-22.153527,-20.466832
2,2024-07-17,2024-08-05,2024-10-29,105,-9.744362,-9.179722
3,2023-07-19,2023-08-18,2023-10-31,105,-8.772029,-8.606643
4,2024-12-12,2025-02-12,2025-02-18,69,-7.044414,-6.973925
5,2024-03-22,2024-04-19,2024-04-30,40,-6.506666,-6.077378


/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:555: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:557: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:565: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").apply(_stats.comp)
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:568: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").last()
/home/renanmogo/mfin-alg

#### Portfolio (Weighted) vs Portolio (Equally-Weighted)

In [13]:
# # Ensure equal_weight_return and portfolio_return are properly aggregated if resampled
# equal_weight_return = equal_weight_return.resample('D').sum() if equal_weight_return.index.freq is None else equal_weight_return
# portfolio_return = portfolio_return.resample('D').sum() if portfolio_return.index.freq is None else portfolio_return

# # Set the name for the equal weight portfolio
# equal_weight_return.name = "Equal Weight Portfolio"

# # Generate Report
# qs.reports.html(
#     portfolio_return,
#     equal_weight_return,
#     rf=risk_free_rate,
#     figsize=(8, 5),
#     output=f'portfolio_vs_equal_weight-{datetime.date(end_date)}.html',
#     title='Portfolio (Weighted) vs Portolio (Equally-Weighted)',
#     benchmark_title="Equal Weight Portfolio",
#     download_filename="portfolio_vs_equal_weight.html" 
# )

# qs.reports.full(
#     portfolio_return, 
#     equal_weight_return,
#     rf=risk_free_rate, 
#     figsize=(8, 5), 
#     title='Portfolio vs Equal Weight',
#     benchmark_title="Equal Weight Portfolio",
#     ) 